# Imports

In [1]:
from tqdm.auto import tqdm

In [2]:
# TODO fix constants
import sys
sys.path.append("../")
import constants

# import minsearch, json

# Knowledgebase setup

In [3]:
#FAQ DATA
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
documents[int(len(documents)/2)]

{'text': "What if there were hundreds of columns? How do you get the columns only with numeric or object data in a more concise way?\ndf.select_dtypes(include=np.number).columns.tolist()\ndf.select_dtypes(include='object').columns.tolist()\nAdded by Gregory Morris",
 'section': '1. Introduction to Machine Learning',
 'question': 'How to select column by dtype',
 'course': 'machine-learning-zoomcamp'}

# ES setup

In [4]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch("http://localhost:9200")
es_client.info()

ObjectApiResponse({'name': 'abe72680beb6', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'k7XZEG03S8WAlYdRIsTWtg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [5]:
es_client.info()['version']['build_hash']

'42f05b9372a9a4a470db3b52817899b99a76ee73'

# Indexing data for ES data 

In [7]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
index_name = "course-questions"
es_client.indices.create(index=index_name, body=index_settings)

In [8]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)  

  0%|          | 0/948 [00:00<?, ?it/s]

# Elastic search

In [9]:
query = "How do I execute a command in a running docker container?"
search_query = {
    # "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

In [10]:
response['hits']['hits']

[{'_index': 'course-questions',
  '_id': '9Gjyb5ABHUU4ElAjTs9Z',
  '_score': 84.17781,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'qGgOcJABHUU4ElAj3dOe',
  '_score': 84.17781,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> 

In [11]:
ml_query = "How do I execute a command in a running docker container?"
ml_search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": ml_query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
                },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
            }
        }
    }


ml_response = es_client.search(index=index_name, body=ml_search_query)
ml_result_docs = []

for ml_hit in ml_response['hits']['hits']:
    ml_result_docs.append(ml_hit['_source'])
ml_result_docs

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from

# Prompt for LLM

In [12]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()


    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

# Gemini setup


In [13]:
GEMINI_API_KEY = constants.GEMINI_API_key
import google.generativeai as genai
genai.configure(api_key=constants.GEMINI_API_key)
model = genai.GenerativeModel('gemini-1.5-pro')

# Answer from LLM

In [14]:
def llm(prompt):
    response = model.generate_content(prompt)
    return response.text

In [15]:
prompt = build_prompt(query, ml_result_docs)
answer = llm(prompt)

In [16]:
print(prompt)
print(len(prompt))

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
section: 5. Deploying Machine Learning Models
question: How do I debug a docker container?
answer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

section: 5. Deploying Machine Learning Models
question: How do I debug a docker container?
answer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
dock

In [18]:
model.count_tokens(prompt)

total_tokens: 373